In [1]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
import threading
import datetime
import traceback
from openpyxl import load_workbook
from openpyxl.styles import Font
import ctypes
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
STOP_FLAG = False
SAVE_INTERVAL = 10        # 10件ごとに自動保存
MAX_TOKENS_PER_ITEM = 10  # 1商品あたり最大トークン使用量
MAX_SECONDS_ALLOWED = 10   # 1商品あたり最大処理時間（秒）
ERROR_WAIT_TIME = 300      # エラー時の待機時間（秒）＝5分

# ============================================================
# スリープ防止（Windows）
# ============================================================
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)
    except Exception:
        pass

def allow_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    except Exception:
        pass

# ============================================================
# Keepa API呼び出し
# ============================================================
def fetch_top_display_price(api_key: str, code: str):
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )

    start_time = time.time()

    try:
        resp = requests.get(url, timeout=MAX_SECONDS_ALLOWED)
        if resp.status_code == 429:
            # トークン枯渇（Too Many Requests）
            return None, None, "トークン枯渇", 0
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0

    if time.time() - start_time > MAX_SECONDS_ALLOWED:
        return None, None, f"処理時間超過（{MAX_SECONDS_ALLOWED}秒）", 0

    if not data or "products" not in data:
        return None, None, "データなし", 0

    products = data["products"]
    if not products:
        return None, None, "商品が見つからない", 0

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # ✅ BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, 0

    # ✅ Prime優先（なければ最初の出品）
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, 0

    hit_count = len(offers)
    if hit_count > 0:
        return title, None, f"価格取得失敗（{hit_count}件ヒット）", 0
    else:
        return title, None, "商品が見つからない", 0

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    prevent_sleep()
    start_button.config(state="disabled")

    try:
        df = pd.read_excel(filepath, header=None)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        allow_sleep()
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_file = os.path.join(desktop_path, f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.see(tk.END)

    try:
        wb = load_workbook(filepath)
        ws = wb.active

        for i, row in df.iterrows():
            excel_row = i + 1
            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中まで保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 保存完了 → {output_file}\n")
                break

            jan = str(row.iloc[0]).strip() if len(row) > 0 else ""
            if jan == "" or jan.lower() == "nan":
                continue

            cell_price = ws.cell(row=excel_row, column=2).value
            if cell_price not in (None, "", " ", "Null"):
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.see(tk.END)
                continue

            title, price, error, _ = fetch_top_display_price(api_key, jan)

            # ✅ トークン枯渇・通信エラー時の待機処理
            if error:
                if "トークン枯渇" in error:
                    log_box.insert(tk.END, f"🪙 {i+1}/{total} {jan} → トークン枯渇を検出。5分間待機します。\n")
                elif "通信エラー" in error:
                    log_box.insert(tk.END, f"⚠️ {i+1}/{total} {jan} → 通信エラー発生。5分間待機します。\n")
                else:
                    log_box.insert(tk.END, f"⚠️ {i+1}/{total} {jan} → {error}（5分待機）\n")
                log_box.see(tk.END)
                time.sleep(ERROR_WAIT_TIME)
                continue  # Excelへは書かずスキップ

            # ✅ タイトルと価格
            ws.cell(row=excel_row, column=3).value = title
            if price is not None:
                ws.cell(row=excel_row, column=2).value = price
                ws.cell(row=excel_row, column=4).value = ""
            else:
                ws.cell(row=excel_row, column=2).value = "Null"
                note = error or ""
                if "価格取得失敗" in note:
                    ws.cell(row=excel_row, column=4, value=note).font = Font(bold=True, color="FF0000")
                elif "商品が見つからない" in note:
                    ws.cell(row=excel_row, column=4, value=note).font = Font(color="808080")
                else:
                    ws.cell(row=excel_row, column=4, value=note)

            # ✅ 10件ごとに保存
            if (i + 1) % SAVE_INTERVAL == 0:
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 {i+1}件完了 → 自動保存しました。\n")
                log_box.see(tk.END)

            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了\n")
            log_box.see(tk.END)
            time.sleep(1.0)

        wb.save(output_file)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")
        allow_sleep()

# ============================================================
# GUI構築
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（Flower版）")
    root.geometry("420x330")
    root.configure(bg="#f5f0e6")
    root.resizable(False, False)

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            allow_sleep()
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    var_topmost = tk.BooleanVar(value=True)
    chk_top = tk.Checkbutton(root, text="常に前面に表示", variable=var_topmost,
                             bg="#f5f0e6", font=("Meiryo", 9),
                             command=lambda: root.attributes("-topmost", var_topmost.get()))
    chk_top.pack(anchor="e", padx=10, pady=(3, 0))
    root.attributes("-topmost", True)

    tk.Label(root, text="Keepa APIキー：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    api_entry = tk.Entry(root, width=55, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイル：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 選択）", bg="white",
                          width=55, height=1, relief="groove", font=("Meiryo", 9))
    file_label.pack(padx=10, pady=2)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 9), width=18).pack(pady=3)

    frame_log = tk.Frame(root, bg="#f5f0e6")
    frame_log.pack(padx=10, pady=(5, 0), fill="both", expand=True)
    log_box = tk.Text(frame_log, height=6, width=55, font=("Meiryo", 8))
    log_box.pack(side="left", fill="both", expand=True)
    scrollbar = tk.Scrollbar(frame_log, command=log_box.yview)
    scrollbar.pack(side="right", fill="y")
    log_box.config(yscrollcommand=scrollbar.set)

    frame_buttons = tk.Frame(root, bg="#f5f0e6")
    frame_buttons.pack(pady=5)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制終了ボタンが押されました。\n")
        log_box.see(tk.END)

    start_button = tk.Button(frame_buttons, text="▶ 開始", bg="#4CAF50", fg="white",
                             font=("Meiryo", 10, "bold"), width=14)
    start_button.pack(side="left", padx=15)

    tk.Button(frame_buttons, text="■ 強制終了", bg="#d9534f", fg="white",
              font=("Meiryo", 10, "bold"), width=14, command=force_stop).pack(side="right", padx=15)

    start_button.config(command=lambda: threading.Thread(
        target=start_process,
        args=(api_entry.get().strip(), getattr(file_label, "filepath", None), log_box, start_button),
        daemon=True
    ).start())

    root.mainloop()

# ============================================================
# エントリーポイント
# ============================================================
if __name__ == "__main__":
    create_gui()


In [ ]:
import requests
import time
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from tkinterdnd2 import DND_FILES, TkinterDnD
import threading
import datetime
import traceback
from openpyxl import load_workbook
from openpyxl.styles import Font
import ctypes
import os

# ============================================================
# 設定値
# ============================================================
DOMAIN_JP = 5
STOP_FLAG = False
SAVE_INTERVAL = 10        # 10件ごとに自動保存
MAX_TOKENS_PER_ITEM = 10  # 1商品あたり最大トークン使用量
MAX_SECONDS_ALLOWED = 10   # 1商品あたり最大処理時間（秒）
ERROR_WAIT_TIME = 300      # エラー時の待機時間（秒）＝5分

# ============================================================
# スリープ防止（Windows）
# ============================================================
ES_CONTINUOUS = 0x80000000
ES_SYSTEM_REQUIRED = 0x00000001

def prevent_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS | ES_SYSTEM_REQUIRED)
    except Exception:
        pass

def allow_sleep():
    try:
        ctypes.windll.kernel32.SetThreadExecutionState(ES_CONTINUOUS)
    except Exception:
        pass

# ============================================================
# Keepa API呼び出し
# ============================================================
def fetch_top_display_price(api_key: str, code: str):
    url = (
        f"https://api.keepa.com/product?key={api_key}"
        f"&domain={DOMAIN_JP}&code={code}"
        "&history=0&offers=20&onlyLiveOffers=0&buybox=1&stats=0"
    )

    start_time = time.time()

    try:
        resp = requests.get(url, timeout=MAX_SECONDS_ALLOWED)
        if resp.status_code == 429:
            # トークン枯渇（Too Many Requests）
            return None, None, "トークン枯渇", 0
        data = resp.json()
    except Exception as e:
        return None, None, f"通信エラー: {e}", 0

    if time.time() - start_time > MAX_SECONDS_ALLOWED:
        return None, None, f"処理時間超過（{MAX_SECONDS_ALLOWED}秒）", 0

    if not data or "products" not in data:
        return None, None, "データなし", 0

    products = data["products"]
    if not products:
        return None, None, "商品が見つからない", 0

    product = products[0]
    title = product.get("title", "")
    stats = product.get("stats") or {}

    # ✅ BuyBox優先
    for key in ("buyBoxPrice", "buyBoxShippingPrice", "current_BUY_BOX_SHIPPING"):
        v = stats.get(key)
        if isinstance(v, (int, float)) and v > 0:
            return title, int(v), None, 0

    # ✅ Prime優先（なければ最初の出品）
    offers = product.get("offers") or []
    order = product.get("liveOffersOrder") or []
    ordered = [offers[i] for i in order if isinstance(i, int) and i < len(offers)]
    if not ordered and offers:
        ordered = offers

    prime_offer = next((o for o in ordered if o.get("isPrime")), None)
    chosen = prime_offer or (ordered[0] if ordered else None)

    if chosen:
        price = chosen.get("price")
        ship = chosen.get("shipping") or 0
        if price and price > 0:
            total = int(price) + int(ship)
            return title, total, None, 0

    hit_count = len(offers)
    if hit_count > 0:
        return title, None, f"価格取得失敗（{hit_count}件ヒット）", 0
    else:
        return title, None, "商品が見つからない", 0

# ============================================================
# メイン処理
# ============================================================
def start_process(api_key, filepath, log_box, start_button):
    global STOP_FLAG
    STOP_FLAG = False
    prevent_sleep()
    start_button.config(state="disabled")

    try:
        df = pd.read_excel(filepath, header=None)
    except Exception as e:
        messagebox.showerror("読込エラー", f"Excelファイルを開けませんでした。\n{e}")
        start_button.config(state="normal")
        allow_sleep()
        return

    total = len(df)
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_file = os.path.join(desktop_path, f"結果_{timestamp}.xlsx")

    log_box.insert(tk.END, f"📘 ファイル読込完了: {filepath}\n🔢 全{total}件の処理を開始します。\n\n")
    log_box.see(tk.END)

    try:
        wb = load_workbook(filepath)
        ws = wb.active

        for i, row in df.iterrows():
            excel_row = i + 1
            if STOP_FLAG:
                log_box.insert(tk.END, "🛑 強制停止を検出しました。途中まで保存します...\n")
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 保存完了 → {output_file}\n")
                break

            jan = str(row.iloc[0]).strip() if len(row) > 0 else ""
            if jan == "" or jan.lower() == "nan":
                continue

            cell_price = ws.cell(row=excel_row, column=2).value
            if cell_price not in (None, "", " ", "Null"):
                log_box.insert(tk.END, f"✅ {i+1}/{total} {jan} → 価格済み（スキップ）\n")
                log_box.see(tk.END)
                continue

            title, price, error, _ = fetch_top_display_price(api_key, jan)

            # ✅ トークン枯渇・通信エラー時の待機処理
           # ✅ トークン枯渇・通信エラー時の待機処理
            if error:
                if "トークン枯渇" in error:
                    log_box.insert(tk.END, f"🪙 {i+1}/{total} {jan} → トークン枯渇を検出。5分間待機します。\n")
                    log_box.see(tk.END)
                    time.sleep(300)  # 5分待機
                    continue
            
                elif "通信エラー" in error:
                    log_box.insert(tk.END, f"⚠️ {i+1}/{total} {jan} → 通信エラー発生。10秒待機します。\n")
                    log_box.see(tk.END)
                    time.sleep(10)  # 10秒待機
                    continue
            
                # 🔹「商品が見つからない」「価格取得失敗」はスルーして続行


            # ✅ タイトルと価格
            ws.cell(row=excel_row, column=3).value = title
            if price is not None:
                ws.cell(row=excel_row, column=2).value = price
                ws.cell(row=excel_row, column=4).value = ""
            else:
                ws.cell(row=excel_row, column=2).value = "Null"
                note = error or ""
                if "価格取得失敗" in note:
                    ws.cell(row=excel_row, column=4, value=note).font = Font(bold=True, color="FF0000")
                elif "商品が見つからない" in note:
                    ws.cell(row=excel_row, column=4, value=note).font = Font(color="808080")
                else:
                    ws.cell(row=excel_row, column=4, value=note)

            # ✅ 10件ごとに保存
            if (i + 1) % SAVE_INTERVAL == 0:
                wb.save(output_file)
                log_box.insert(tk.END, f"💾 {i+1}件完了 → 自動保存しました。\n")
                log_box.see(tk.END)

            log_box.insert(tk.END, f"🕐 {i+1}/{total} 完了\n")
            log_box.see(tk.END)
            time.sleep(1.0)

        wb.save(output_file)
        log_box.insert(tk.END, f"\n🎉 完了！結果を「{output_file}」に保存しました。\n")
        messagebox.showinfo("完了", f"処理が完了しました！\n結果ファイル: {output_file}")

    except Exception as e:
        log_box.insert(tk.END, f"⚠️ エラー発生: {e}\n")
        log_box.insert(tk.END, traceback.format_exc())
        messagebox.showerror("エラー", f"処理中に問題が発生しました。\n{output_file}")

    finally:
        start_button.config(state="normal")
        allow_sleep()

# ============================================================
# GUI構築
# ============================================================
def create_gui():
    root = TkinterDnD.Tk()
    root.title("Keepa価格取得ツール（Flower版）")
    root.geometry("420x330")
    root.configure(bg="#f5f0e6")
    root.resizable(False, False)

    def on_close():
        if messagebox.askyesno("確認", "本当に終了しますか？"):
            allow_sleep()
            root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    var_topmost = tk.BooleanVar(value=True)
    chk_top = tk.Checkbutton(root, text="常に前面に表示", variable=var_topmost,
                             bg="#f5f0e6", font=("Meiryo", 9),
                             command=lambda: root.attributes("-topmost", var_topmost.get()))
    chk_top.pack(anchor="e", padx=10, pady=(3, 0))
    root.attributes("-topmost", True)

    tk.Label(root, text="Keepa APIキー：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    api_entry = tk.Entry(root, width=55, show="*")
    api_entry.pack(padx=10)

    tk.Label(root, text="Excelファイル：", bg="#f5f0e6", font=("Meiryo", 10, "bold")).pack(anchor="w", padx=10, pady=2)
    file_label = tk.Label(root, text="（ここにドラッグ＆ドロップ または 選択）", bg="white",
                          width=55, height=1, relief="groove", font=("Meiryo", 9))
    file_label.pack(padx=10, pady=2)

    def drop_file(event):
        filepath = event.data.strip("{}")
        file_label.config(text=filepath)
        file_label.filepath = filepath

    def select_file():
        filepath = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx *.xls")])
        if filepath:
            file_label.config(text=filepath)
            file_label.filepath = filepath

    file_label.drop_target_register(DND_FILES)
    file_label.dnd_bind('<<Drop>>', drop_file)
    tk.Button(root, text="ファイルを選択", command=select_file, font=("Meiryo", 9), width=18).pack(pady=3)

    frame_log = tk.Frame(root, bg="#f5f0e6")
    frame_log.pack(padx=10, pady=(5, 0), fill="both", expand=True)
    log_box = tk.Text(frame_log, height=6, width=55, font=("Meiryo", 8))
    log_box.pack(side="left", fill="both", expand=True)
    scrollbar = tk.Scrollbar(frame_log, command=log_box.yview)
    scrollbar.pack(side="right", fill="y")
    log_box.config(yscrollcommand=scrollbar.set)

    frame_buttons = tk.Frame(root, bg="#f5f0e6")
    frame_buttons.pack(pady=5)

    def force_stop():
        global STOP_FLAG
        STOP_FLAG = True
        log_box.insert(tk.END, "\n🛑 強制終了ボタンが押されました。\n")
        log_box.see(tk.END)

    start_button = tk.Button(frame_buttons, text="▶ 開始", bg="#4CAF50", fg="white",
                             font=("Meiryo", 10, "bold"), width=14)
    start_button.pack(side="left", padx=15)

    tk.Button(frame_buttons, text="■ 強制終了", bg="#d9534f", fg="white",
              font=("Meiryo", 10, "bold"), width=14, command=force_stop).pack(side="right", padx=15)

    start_button.config(command=lambda: threading.Thread(
        target=start_process,
        args=(api_entry.get().strip(), getattr(file_label, "filepath", None), log_box, start_button),
        daemon=True
    ).start())

    root.mainloop()

# ============================================================
# エントリーポイント
# ============================================================
if __name__ == "__main__":
    create_gui()


Exception in thread Thread-9 (start_process):
Traceback (most recent call last):
  File "C:\Users\taku5\AppData\Local\Temp\ipykernel_5812\4249337151.py", line 194, in start_process
  File "C:\Users\taku5\anaconda3\Lib\tkinter\__init__.py", line 3808, in insert
    self.tk.call((self._w, 'insert', index, chars) + args)
RuntimeError: main thread is not in main loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\taku5\AppData\Local\Temp\ipykernel_5812\4249337151.py", line 198, in start_process
  File "C:\Users\taku5\anaconda3\Lib\tkinter\__init__.py", line 3808, in insert
    self.tk.call((self._w, 'insert', index, chars) + args)
RuntimeError: main thread is not in main loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\taku5\anaconda3\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\taku5\anaconda3\Lib\thre

In [3]:
import requests

API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"

url = f"https://api.keepa.com/token?key={API_KEY}"
response = requests.get(url)
data = response.json()

print("🪙 現在のトークン残量:", data.get("tokensLeft", "取得失敗"))
print("🔄 トークン再充填まで:", data.get("refillIn", "不明"), "ミリ秒後")
print("💧 補充レート:", data.get("refillRate", "不明"), "トークン/分")


🪙 現在のトークン残量: 265
🔄 トークン再充填まで: 9824 ミリ秒後
💧 補充レート: 5 トークン/分


In [ ]:
5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j